In [26]:
import importlib
from tools import network_tools as network_tools
from tools import firewall_tools as firewall_tools
importlib.reload(network_tools)
importlib.reload(firewall_tools)

<module 'tools.firewall_tools' from '/home/c0ff3k1ll3r/Desktop/Thesis/AI_Agent_for_honeypot_operations/HoneypotAgentApp/MultiAgentArchitecture/tools/firewall_tools.py'>

In [48]:
alerts = await network_tools.get_fast_alerts(time_window=1)
alerts

{'security_events': {'alerts': [{'message': '[**] [1:3400001:1] POSSBL PORT SCAN (NMAP -sS) [**] [Classification: Attempted Information Leak] [Priority: 2] {TCP} 192.168.100.2:37389 -> 172.20.0.6:443',
    'timestamp': '2025-08-30T16:41:56.379315+02:00'},
   {'message': '[**] [1:3400001:1] POSSBL PORT SCAN (NMAP -sS) [**] [Classification: Attempted Information Leak] [Priority: 2] {TCP} 192.168.100.2:37389 -> 172.20.0.17:443',
    'timestamp': '2025-08-30T16:41:56.582188+02:00'},
   {'message': '[**] [1:3400001:1] POSSBL PORT SCAN (NMAP -sS) [**] [Classification: Attempted Information Leak] [Priority: 2] {TCP} 192.168.100.2:37389 -> 172.20.0.32:443',
    'timestamp': '2025-08-30T16:41:56.582263+02:00'},
   {'message': '[**] [1:3400001:1] POSSBL PORT SCAN (NMAP -sS) [**] [Classification: Attempted Information Leak] [Priority: 2] {TCP} 192.168.100.2:37389 -> 172.20.0.49:443',
    'timestamp': '2025-08-30T16:41:56.584509+02:00'},
   {'message': '[**] [1:3400001:1] POSSBL PORT SCAN (NMAP -s

In [20]:
network_tools.get_docker_containers()

{'honeypot_config': [{'id': '0f6132a05b65',
   'name': 'cve-2021-22205-gitlab-1-proxy',
   'image': 'nginx:alpine',
   'status': 'running',
   'created': '2025-08-29T07:53:01.572843942Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.10'},
  {'id': '814750f7a17a',
   'name': 's2-057-struts2-1',
   'image': 'vulhub/struts2:2.3.34-showcase',
   'status': 'running',
   'created': '2025-08-29T07:53:01.124882815Z',
   'ports': ['8080/tcp'],
   'ip_address': '172.20.0.3'},
  {'id': '4aa1d79e654c',
   'name': 'unauthorized-rce-docker-1',
   'image': 'unauthorized-rce-docker:latest',
   'status': 'running',
   'created': '2025-08-29T07:53:00.799705245Z',
   'ports': ['2375/tcp', '2376/tcp'],
   'ip_address': '172.20.0.2'},
  {'id': 'be171d7f05eb',
   'name': 'xdebug-rce-xdebug2-1',
   'image': 'vulhub/php:7.1-xdebug',
   'status': 'running',
   'created': '2025-08-29T07:52:58.77089242Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.7'}]}

In [46]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
5    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
6    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [5]:
await firewall_tools.remove_firewall_rule([1])

INFO:tools.firewall_tools:Removing firewall rules: [1]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.3           tcp dpt:8080']}

In [47]:
honeypot_ip = '172.20.0.7' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.7 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.7 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.7 -> 192.168.100.2/24 (tcp)'}